# Comment/Annotation Nanopublication Generator

Creates paper annotation nanopublications from a JSON configuration file.

**Template:** [Paper Annotation Template](https://w3id.org/np/RA24onqmqTMsraJ7ypYFOuckmNWpo4Zv5gsLqhXt7xYPU)

## Comment Nanopublications
Annotate papers with:
- Exact quotations from papers
- Your interpretation/comment
- Link to the paper being annotated

In [7]:
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_literal, PREFIXES
)

In [8]:
# Comment-specific constants
COMMENT_TEMPLATE = "https://w3id.org/np/RA24onqmqTMsraJ7ypYFOuckmNWpo4Zv5gsLqhXt7xYPU"
CITO_CITES = "http://purl.org/spar/cito/cites"
CITO_HAS_QUOTED_TEXT = "http://purl.org/spar/cito/hasQuotedText"
CITO_QUOTES = "http://purl.org/spar/cito/quotes"
RDFS_COMMENT = "http://www.w3.org/2000/01/rdf-schema#comment"
DCT_IS_PART_OF = "http://purl.org/dc/terms/isPartOf"

class CommentNanopubGenerator(NanopubGenerator):
    """Generator for comment/annotation nanopublications matching Nanodash format."""
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': COMMENT_TEMPLATE
        }
        super().__init__(merged_config)
        
        # Comment template uses only cito:cites as type
        self.add_nanopub_type(CITO_CITES)
        
        # isPartOf goes in assertion for Comment (links the paper being annotated)
        self.is_part_of_in_assertion = True
    
    def generate_assertion(self) -> str:
        """Generate the comment assertion graph matching Nanodash signed format."""
        paper_doi = self.config['paper_doi']
        quotation = self.config['quotation']
        comment = self.config['comment']
        creator_orcid = self.config.get('creator_orcid', '0000-0000-0000-0000')
        is_part_of = self.config.get('is_part_of')
        
        # Format paper URI
        paper_uri = paper_doi if paper_doi.startswith('http') else f'https://doi.org/{paper_doi}'
        orcid_uri = f'https://orcid.org/{creator_orcid}'
        
        # Truncate label if needed
        label_text = quotation[:50] + '...' if len(quotation) > 50 else quotation
        self.config['label'] = f'Paper annotation: {label_text}'
        
        lines = [f'{self.sub_prefix}:assertion {{']
        
        # Paper with quoted text and comment
        lines.append(f'  <{paper_uri}>')
        lines.append(f'    <{CITO_HAS_QUOTED_TEXT}> {make_literal(quotation)};')
        
        # Add isPartOf for paper (paper is part of systematic review)
        if is_part_of and is_part_of.get('uri'):
            lines.append(f'    <{DCT_IS_PART_OF}> <{is_part_of["uri"]}>;')
        
        lines.append(f'    <{RDFS_COMMENT}> {make_literal(comment)} .')
        
        # ORCID quotes the paper
        lines.append(f'  <{orcid_uri}> <{CITO_QUOTES}> <{paper_uri}> .')
        
        lines.append('}')
        return '\n'.join(lines)

In [9]:
# Configuration
CONFIG_FILE = "../config/vbae208/vbae208_comment.json"  # Change this to use different config
CONFIG_FILE = "../config/clenet2025/clenet2025_comment.json"  # Change this to use different config
OUTPUT_DIR = "../output/comment"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [10]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"Number of comment nanopublications to generate: {len(config['nanopublications'])}")
if config['metadata'].get('is_part_of'):
    print(f"Part of: {config['metadata']['is_part_of']['label']}")
print()

for i, np_config in enumerate(config['nanopublications'], 1):
    print(f"{i}. {np_config['quotation'][:60]}...")

Source paper: Addressing Ecological Challenges from a Quantum Computing Perspective
Number of comment nanopublications to generate: 4
Part of: Quantum Computing Applications for Biodiversity Research - Systematic Review

1. Quantum computers will have a significant impact on ecology ...
2. For problems where a classical computer might require millio...
3. Efficiently translating these classical datasets into quantu...
4. NISQ devices represent the current generation of quantum com...


In [11]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = CommentNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

Generated: ../output/comment/comment_clenet_impact.trig
Generated: ../output/comment/comment_clenet_speedup.trig
Generated: ../output/comment/comment_clenet_limitation.trig
Generated: ../output/comment/comment_clenet_nisq.trig

Total generated: 4 nanopublications


In [12]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

Preview of ../output/comment/comment_clenet_impact.trig:

@prefix this: <https://w3id.org/np/RA6e037db988d0823fc21c23ba1cdcf6f820917859000> .
@prefix sub: <https://w3id.org/np/RA6e037db988d0823fc21c23ba1cdcf6f820917859000/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:Head {
  this: a np:Nanopublication ;
    np:hasAssertion sub:assertion ;
    np:hasProvenance sub:provenance ;
    np:hasPublicationInfo sub:pubinfo .
}

sub:assertion {
  <https://doi.org/10.48550/arXiv.2504.03866>
    <http://purl.org/spar/cito/hasQuotedText> "Quantum computers will ha

## JSON Config Structure

```json
{
  "metadata": {
    "source_paper": { "title": "...", "doi": "..." },
    "creator_orcid": "0000-0000-0000-0000",
    "creator_name": "Your Name",
    "is_part_of": {
      "uri": "https://w3id.org/np/YOUR-REVIEW-URI",
      "label": "Your Systematic Review Title"
    }
  },
  "nanopublications": [
    {
      "id": "comment_example_01",
      "paper_doi": "10.xxxx/xxxxx",
      "quotation": "Exact quote from the paper...",
      "comment": "Your interpretation and relevance to the review..."
    }
  ]
}
```

The `is_part_of` creates a `dct:isPartOf` triple in the **assertion** linking the paper being annotated to your systematic review.